In [1]:
import time
from typing import Dict, Optional, Any

from ray import train, tune
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch

from functools import partial

In [2]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation == "relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost


def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        train.report({"iterations": step, "mean_loss": score})

In [ ]:
# search_space = {
#     "steps": 100,
#     "width": tune.uniform(0, 20),
#     "height": tune.uniform(-100, 100),
#     "activation": tune.choice(["relu", "tanh"]),
# }


def define_by_run_func(trial) -> Optional[Dict[str, Any]]:
    """creates the search space.
    This function should either return None or a dict with constant values.
    """
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("width", 0, 1)
        trial.suggest_float("height", 0, 1)
    else:
        trial.suggest_float("width", 100, 200)
        trial.suggest_float("height", 100, 200)

    # Return all constants in a dictionary.
    return {"steps": 100}


searcher = OptunaSearch(space=define_by_run_func, metric="mean_loss", mode="min")
algo = ConcurrencyLimiter(searcher, max_concurrent=4)
num_samples = 2

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=algo,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

In [3]:
def define_by_run_func(trial, i) -> Optional[Dict[str, Any]]:
    print(f"Extra parameter passed: {i}")
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    if activation == "relu":
        trial.suggest_float("width", 0, 1)
        trial.suggest_float("height", 0, 1)
    else:
        trial.suggest_float("width", 100, 200)
        trial.suggest_float("height", 100, 200)

    return {"steps": 100}


# Usage
extra_param_value = "example"
custom_define_by_run_func = partial(define_by_run_func, i=extra_param_value)

searcher = OptunaSearch(space=custom_define_by_run_func, metric="mean_loss", mode="min")

algo = ConcurrencyLimiter(searcher, max_concurrent=4)
num_samples = 2

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=algo,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

Extra parameter passed: example
Extra parameter passed: example


2024-06-20 17:15:32,538	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/adamova/ray_results/objective_2024-06-20_17-15-04' in 0.0135s.
2024-06-20 17:15:32,544	INFO tune.py:1041 -- Total run time: 23.80 seconds (19.38 seconds for the tuning loop).
